In [1]:
import pandas, os, re, numpy, matplotlib, datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import gridspec
import matplotlib.font_manager as fm
import matplotlib

# 函數庫
1. 獨立樣本檢定函數，自動化判別，並將顯著結果輸出。

In [ ]:
def os_path(list) : 
    os_path = os.path.dirname(os.path.dirname(os.getcwd()))
    for i in list : 
        os_path = os.path.join(os_path, i)
    return os_path

In [2]:
import scipy.stats as stats

# 執行 Shapiro-Wilk_test、(常態性檢定)
def Shapiro_Wilk_test(df) :
    statistic, p_value = stats.shapiro(df)
    if p_value <=0.05 :
        return "pass"               
    else : 
        return "reject"


# 執行變異數同質性檢定
def levene_test(df_list) :
    statistic, p_value = stats.levene(df_list[0], df_list[1])
    if p_value <=0.05 :       
        return "pass"
    else :
        return "reject"

# 執行獨立樣本 T 檢定
def ttest_ind_test(df_list) :
    statistic, p_value = stats.ttest_ind(df_list[0], df_list[1])
    if p_value <=0.05 :
        return "pass"
    else :
        return "reject"

# 載入數據

In [2]:
df = pandas.read_csv(os.path.join(os.getcwd(), "database_clean", "database_clean.csv"))

# df.head(3)

# 敘述統計部分

## 由北到南縣市列表

In [6]:
western_counties = ['基隆市', '台北市', '新北市', '桃園市', '新竹市', '新竹縣', '苗栗縣', '台中市', '彰化縣', '南投縣', '雲林縣', '嘉義市', '嘉義縣', '台南市', '高雄市', '屏東縣']

## 能否遠端工作，變數建立。

In [3]:
# 建立圖片存放位置
output_folder_path = os.path.join(os_path(["picture", "遠端工作各縣市比例"]))
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
else:
    pass

In [4]:
df["remote_work"] = numpy.where((df["work_name"].str.contains("遠端")) | (df["requirements"].str.contains("遠端")), 1, 0)

# 排除在疫情之前，就是遠端工作的職業，如直播主，則可以得到各縣市是在疫情後出現的遠端工作比例。
# 接著如果台北市的遠端工作機會，真的具有推動人口外移的真實作用的話，則在台北市的遠端工作比例，應當比起北漂青年居住地(桃園 等)，在遠端工作的比例更高。
df["requirements"] = df["requirements"].astype(str)
remote_work_area_counts =  df[(df["remote_work"] == 1) & ~(df["work_name"].str.contains("直播")) & ~(df["requirements"].str.contains("直播"))]["area"].value_counts()

remote_work_area_counts_dict = {}
for i in remote_work_area_counts.keys() : 
    remote_work_area_counts_dict[i] = remote_work_area_counts[i]/len(df[df["area"] == i])

matplotlib.rcParams["font.family"] = "simsun"
plt.bar(remote_work_area_counts_dict.keys(), remote_work_area_counts_dict.values())
plt.xlabel("縣市別")
plt.xticks(rotation= 45)
plt.title("遠端工作比例")
plt.savefig(os_path(["picture", "遠端工作各縣市比例", "bar_遠端工作各縣市比例.png"]), dpi=200, bbox_inches = "tight")
# plt.show()
plt.close()

In [7]:
# 由北到南的折線圖版本
western_counties_values = []
for counties in western_counties : 
    western_counties_values.append(remote_work_area_counts_dict[counties]) 

plt.plot(range(len(western_counties)), western_counties_values)
plt.xlabel("縣市別(不含東部)")
plt.xticks(range(len(western_counties)), western_counties, rotation= 45)
plt.title("遠端工作比例")
plt.savefig(os_path(["picture", "遠端工作各縣市比例", "由北到南_遠端工作各縣市比例.png"]), dpi=200, bbox_inches = "tight")
# plt.show()
plt.close()

In [8]:
remote_work_area_counts_dict

{'台北市': 0.005416770086580719,
 '新北市': 0.0032131980912644175,
 '台中市': 0.002576025786678644,
 '桃園市': 0.0028055192857394926,
 '高雄市': 0.0024405125076266015,
 '台南市': 0.0015818689062095544,
 '新竹市': 0.002341681938505396,
 '彰化縣': 0.002270630809621798,
 '新竹縣': 0.0012782694198623403,
 '屏東縣': 0.0029641606036108865,
 '雲林縣': 0.0028666264333132166,
 '嘉義市': 0.003971034804952114,
 '宜蘭縣': 0.0016005121638924455,
 '花蓮縣': 0.0024330900243309003,
 '南投縣': 0.0011609907120743034,
 '嘉義縣': 0.0005898545025560362,
 '苗栗縣': 0.0003498542274052478,
 '基隆市': 0.0007010164738871364,
 '台東縣': 0.0004975124378109452,
 '澎湖縣': 0.0017421602787456446}

In [9]:
# 各縣市遠端工作比例 Excel檔建立。
df_for_list = []
for key in remote_work_area_counts_dict.keys() : 
    df_for_list.append({"縣市別" : key, "遠端工作比例" : remote_work_area_counts_dict[key]})

df_for = pandas.DataFrame(df_for_list)

In [11]:
df_for.to_excel(os_path(["檢定資料","遠端工作比例.xlsx"]), index= False)

## 相同類別之工作在各縣市的工作機會差異

### 相同類別之職業在不同縣市的機會差異(總數)

#### 全部縣市之長條圖

In [10]:
# 判斷 輸出資料夾是否存在
output_folder_path = os_path(["picture", "same_profession_county", "bar"])
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
else:
    pass

In [11]:
for Profession_classification in list(dict.fromkeys(df["Profession_classification_chinese"])) : 
    # print(Profession_classification)
    same_profession_dict = {}
    for area in list(dict.fromkeys(df["area"])) : 
        same_profession_dict[area] = len(df[(df["area"] == area) & (df["Profession_classification_chinese"] == Profession_classification)])

    # 繪致折線圖
    plt.bar(same_profession_dict.keys(), same_profession_dict.values())    
    plt.xlabel("縣市別")
    plt.xticks(rotation= 45)
    plt.title(Profession_classification)

    plt.savefig(os.path.join(output_folder_path, f"{Profession_classification}.png"), dpi=200, bbox_inches = "tight")
    # plt.show()
    plt.close()
 

#### 由北到南(不含東部)之折線圖

In [12]:
# 判斷 輸出資料夾是否存在
output_folder_path = os_path(["picture", "same_profession_county", "plot"])
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
else:
    pass

In [13]:
for Profession_classification in list(dict.fromkeys(df["Profession_classification_chinese"])) : 
    same_profession_dict = {}
    for area in western_counties : 
        same_profession_dict[area] = len(df[(df["area"] == area) & (df["Profession_classification_chinese"] == Profession_classification)])

    # 繪致折線圖
    plt.plot(range(len(same_profession_dict.keys())), same_profession_dict.values())    
    plt.xlabel("由北到南（不含東部）")
    plt.xticks(range(len(western_counties)), western_counties, rotation= 45)
    plt.title(Profession_classification)
    plt.savefig(os.path.join(output_folder_path, f"{Profession_classification}.png"), dpi=200, bbox_inches = "tight")
    # plt.show()
    plt.close()


## 相同類別之職業平均薪資由北到南之趨勢圖

北部區域：包括臺北市、新北市、基隆市、新竹市、桃園市、新竹縣及宜蘭縣。

中部區域：包括臺中市、苗栗縣、彰化縣、南投縣及雲林縣。

南部區域：包括高雄市、臺南市、嘉義市、嘉義縣、屏東縣及澎湖縣。

### 全部縣市之長條圖

In [14]:
# 判斷 輸出資料夾是否存在
output_folder_path = os_path(["picture", "same_profession_salary_county", "Mix", "bar"])
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
else:
    pass

In [15]:
# 最小值
for Profession_classification in list(dict.fromkeys(df["Profession_classification_chinese"])) : 
    # print(Profession_classification)
    same_profession_dict = {}
    for area in list(dict.fromkeys(df["area"])) : 
        same_profession_dict[area] = df[(df["area"] == area) & (df["Profession_classification_chinese"] == Profession_classification) & (df["month_salary_classfication"] == 1)]["mix_salary"].mean()

    # 繪致長條圖
    plt.bar(same_profession_dict.keys(), same_profession_dict.values())    
    plt.xlabel("縣市別")
    plt.xticks(rotation= 45)
    plt.title(f"{Profession_classification}_最小值")

    plt.savefig(os.path.join(output_folder_path, f"{Profession_classification}_最小值.png"), dpi=200, bbox_inches = "tight")
    # plt.show()
    plt.close()


In [16]:
# 判斷 輸出資料夾是否存在
output_folder_path = os_path(["picture", "same_profession_salary_county", "Max", "bar"])
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
else:
    pass

In [17]:
# 最大值
for Profession_classification in list(dict.fromkeys(df["Profession_classification_chinese"])) : 
    # print(Profession_classification)
    same_profession_dict = {}
    for area in list(dict.fromkeys(df["area"])) : 
        same_profession_dict[area] = df[(df["area"] == area) & (df["Profession_classification_chinese"] == Profession_classification) & (df["month_salary_classfication"] == 1)]["max_salary"].mean()

    # 繪致長條圖
    plt.bar(same_profession_dict.keys(), same_profession_dict.values())    
    plt.xlabel("縣市別")
    plt.xticks(rotation= 45)
    plt.title(f"{Profession_classification}_最大值")

    plt.savefig(os.path.join(output_folder_path, f"{Profession_classification}_最大值.png"), dpi=200, bbox_inches = "tight")
    # plt.show()
    plt.close()

### 由北到南(不含東部)之折線圖

In [18]:
# 判斷 輸出資料夾是否存在
output_folder_path = os_path(["picture", "same_profession_salary_county", "Mix", "plot"])
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
else:
    pass

In [19]:
# 起薪最小值平均
for Profession_classification in list(dict.fromkeys(df["Profession_classification_chinese"])) : 
    same_profession_dict = {}
    for area in western_counties : 
        same_profession_dict[area] = df[(df["area"] == area) & (df["Profession_classification_chinese"] == Profession_classification) & (df["month_salary_classfication"] == 1)]["mix_salary"].mean()

    # 繪致折線圖
    plt.plot(range(len(same_profession_dict.keys())), same_profession_dict.values())
    plt.xlabel("由北到南（不含東部）")
    plt.xticks(range(len(western_counties)), western_counties, rotation= 45)
    plt.title(f"起薪最小值_{Profession_classification}")

    plt.savefig(os.path.join(output_folder_path, f"{Profession_classification}_最小值.png"), dpi=200, bbox_inches = "tight")
    # plt.show()
    plt.close()



In [20]:
# 判斷 輸出資料夾是否存在
output_folder_path = os_path(["picture", "same_profession_salary_county", "Max", "plot"])
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
else:
    pass

In [21]:
# 起薪最大值
for Profession_classification in list(dict.fromkeys(df["Profession_classification_chinese"])) : 
    same_profession_dict = {}
    for area in western_counties : 
        same_profession_dict[area] = df[(df["area"] == area) & (df["Profession_classification_chinese"] == Profession_classification) & (df["month_salary_classfication"] == 1)]["max_salary"].mean()

    # 繪致折線圖
    plt.plot(range(len(same_profession_dict.keys())), same_profession_dict.values())
    plt.xlabel("由北到南（不含東部）")
    plt.xticks(range(len(western_counties)), western_counties, rotation= 45)
    plt.title(f"{Profession_classification}_起薪最大值")
    plt.savefig(os.path.join(output_folder_path, f"{Profession_classification}_最大值.png"), dpi=200, bbox_inches = "tight")
    # plt.show()
    plt.close()

## 相同職業在各縣市競爭壓力差異

### 不同職業競爭壓力圖 (以工作機會 / 勞動人口數)
網址 : https://winsta.dgbas.gov.tw/DgbasWeb/ZWeb/ShowQuery.aspx?Z=

資料下載日期 : 2023/5/27

行政院主計總處國勢普查處

In [12]:
# 四月各縣市人口數
df_Registered_Population = pandas.read_excel(os_path(["database", "勞動人口數(千人)_按縣市別.xlsx"]), header=1)

df_Registered_Population = df_Registered_Population.iloc[[1]]

# 移除第一欄
df_Registered_Population = df_Registered_Population.drop(df_Registered_Population.columns[0], axis=1)

rename_dict = {}
for i in df_Registered_Population.columns : 
    if "臺" in i : 
        rename_dict[i] = i.replace("臺", "台")

df_Registered_Population= df_Registered_Population.rename(columns= rename_dict)

df_Registered_Population_dict = {}
for i in df_Registered_Population.columns : 
    df_Registered_Population_dict[i] = int(df_Registered_Population[i].values)
                                                  
df_Registered_Population_dict
# 人口數預處理
# 原始的縣市字典






c:\Users\Sigolon\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


{'新北市': 2043,
 '台北市': 1212,
 '桃園市': 1185,
 '台中市': 1465,
 '台南市': 989,
 '高雄市': 1398,
 '宜蘭縣': 223,
 '新竹縣': 290,
 '苗栗縣': 277,
 '彰化縣': 639,
 '南投縣': 254,
 '雲林縣': 353,
 '嘉義縣': 267,
 '屏東縣': 417,
 '台東縣': 106,
 '花蓮縣': 155,
 '澎湖縣': 50,
 '基隆市': 182,
 '新竹市': 221,
 '嘉義市': 127,
 '金門縣': 30}

#### 長條圖

In [23]:
# 判斷 輸出資料夾是否存在
output_folder_path = os_path(["picture", "Competition_Pressure_Among_Different_Occupations_Chart", "bar"])
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
else:
    pass

In [24]:
for Profession_classification in list(dict.fromkeys(df["Profession_classification_chinese"])) : 
    # print(Profession_classification)
    same_profession_dict = {}
    for area in list(dict.fromkeys(df["area"])) : 
        same_profession_dict[area] = len(df[(df["area"] == area) & (df["Profession_classification_chinese"] == Profession_classification)])/((df_Registered_Population_dict[area])*1000)
        # 工作機會 / 勞動人口總數
    
    # 繪致長條圖
    plt.bar(same_profession_dict.keys(), same_profession_dict.values())    
    plt.xlabel("縣市別")
    plt.xticks(rotation= 45)
    plt.title(f"{Profession_classification}")
    plt.savefig(os.path.join(output_folder_path, f"{Profession_classification}_工作機會與勞動人口之比值.png"), dpi=200, bbox_inches = "tight")
    # plt.show()
    plt.close()



#### 折線圖

In [25]:
# 判斷 輸出資料夾是否存在
output_folder_path = os_path(["picture", "Competition_Pressure_Among_Different_Occupations_Chart", "plot"])
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
else:
    pass

In [26]:
for Profession_classification in list(dict.fromkeys(df["Profession_classification_chinese"])) : 
    # print(Profession_classification)
    same_profession_dict = {}
    for area in western_counties : 
        same_profession_dict[area] = len(df[(df["area"] == area) & (df["Profession_classification_chinese"] == Profession_classification)])/((df_Registered_Population_dict[area])*1000)
        # 工作機會 / 勞動人口總數
    
    # 繪致長條圖
    plt.plot(range(len(same_profession_dict.keys())), same_profession_dict.values())
    plt.xlabel("由北到南（不含東部）")
    plt.xticks(range(len(western_counties)), western_counties, rotation= 45)
    plt.title(f"{Profession_classification}")
    plt.savefig(os.path.join(output_folder_path, f"{Profession_classification}_工作機會與勞動人口之比值.png"), dpi=200, bbox_inches = "tight")
    # plt.show()
    plt.close()



#### Tableau 使用資料匯出，各縣市競爭壓力。

In [13]:
# Tableau 使用資料匯出，各縣市競爭壓力。
df_for_list = []
for Profession_classification in list(dict.fromkeys(df["Profession_classification_chinese"])) : 
    for area in list(dict.fromkeys(df["area"])) : 
        competitive_pressure = len(df[(df["area"] == area) & (df["Profession_classification_chinese"] == Profession_classification)])/((df_Registered_Population_dict[area])*1000)
        df_for_list.append({"縣市別" : area, "職業類型" : Profession_classification, "競爭壓力值" : competitive_pressure})

df_for = pandas.DataFrame(df_for_list)
df_for

,縣市別,職業類型,競爭壓力值
0,台北市,經營、人資類,0.005958
1,新北市,經營、人資類,0.001818
2,基隆市,經營、人資類,0.000819
3,桃園市,經營、人資類,0.001819
4,台南市,經營、人資類,0.001511
...,...,...,...
373,澎湖縣,其他類,0.000340
374,苗栗縣,其他類,0.000690
375,嘉義縣,其他類,0.000607
376,嘉義市,其他類,0.000724


In [19]:
df_for.to_excel(os_path(["檢定資料", "競爭力比例.xlsx"]), index= False)